### Vector stores and retrievers

This video tutorial will familiarize you with langchain's vector store and retriever abstractions. These abstractions are designed to support retrieval of data-- from (vector) databases and other sources -- for intergration with llm workflows. They are important for applications that fetch data to be reasoned over as part of model inference, as in the case of retrieval-augmented generation.

we will cover

 - Documents 
 - vector stores 
 - retriever 
 

#### <b> Documents </b>

Langchain implements a Document abstraction, which is intended to represent a unit of text and associated metadata.it has two attributes: 

- page_content : a string representing the content;
- metadata: a dict containing arbitrary metadata. 
The metadata attribute can capture information about the source of the document, its relationship to other documents, and other information. Note that an individual Documents object often represents a chunk of a larger docments.

Let's generate some sample docments: 

In [3]:
from langchain_core.documents import Document
from rich import print
document=[
    Document(
        page_content='Dog are greate companions, know for their loyalty and friendliness.',
        metadata={"source":"mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source":"mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source":"fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source":"bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source":"mammal-pets-doc"},
    ),
]

In [4]:
print(document)

[
    Document(
        metadata={'source': 'mammal-pets-doc'},
        page_content='Dog are greate companions, know for their loyalty and friendliness.'
    ),
    Document(
        metadata={'source': 'mammal-pets-doc'},
        page_content='Cats are independent pets that often enjoy their own space.'
    ),
    Document(
        metadata={'source': 'fish-pets-doc'},
        page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'
    ),
    Document(
        metadata={'source': 'bird-pets-doc'},
        page_content='Parrots are intelligent birds capable of mimicking human speech.'
    ),
    Document(
        metadata={'source': 'mammal-pets-doc'},
        page_content='Rabbits are social animals that need plenty of space to hop around.'
    )
]

In [5]:
## vector stores 
from langchain_chroma import Chroma
import os 
from dotenv import load_dotenv 
load_dotenv()
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [9]:
from langchain_openai import ChatOpenAI
from langchain_ollama import OllamaEmbeddings

llm=ChatOpenAI(model="gpt-5")
embedding=OllamaEmbeddings(model="llama3")

In [11]:
vectorStore=Chroma.from_documents(document,embedding=embedding)

In [13]:
print(vectorStore)

<langchain_chroma.vectorstores.Chroma object at 0x123fe7400>

In [15]:
print(vectorStore.similarity_search("cat"))

[
    Document(
        id='a7fe7797-bfab-4d53-962a-ae2ba3b791c6',
        metadata={'source': 'mammal-pets-doc'},
        page_content='Dog are greate companions, know for their loyalty and friendliness.'
    ),
    Document(
        id='b2f84a37-544b-41b1-9627-f691b50072ad',
        metadata={'source': 'mammal-pets-doc'},
        page_content='Cats are independent pets that often enjoy their own space.'
    ),
    Document(
        id='34ac91a2-3ee2-4ca9-90a7-361cf5705739',
        metadata={'source': 'fish-pets-doc'},
        page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'
    ),
    Document(
        id='fa2a432a-58fa-4aeb-98db-21875d1c07cf',
        metadata={'source': 'bird-pets-doc'},
        page_content='Parrots are intelligent birds capable of mimicking human speech.'
    )
]

In [17]:
## Async query 
await vectorStore.asimilarity_search("cat")

[Document(id='a7fe7797-bfab-4d53-962a-ae2ba3b791c6', metadata={'source': 'mammal-pets-doc'}, page_content='Dog are greate companions, know for their loyalty and friendliness.'),
 Document(id='b2f84a37-544b-41b1-9627-f691b50072ad', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='34ac91a2-3ee2-4ca9-90a7-361cf5705739', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
 Document(id='fa2a432a-58fa-4aeb-98db-21875d1c07cf', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

In [18]:
vectorStore.similarity_search_with_score("cat")

[(Document(id='a7fe7797-bfab-4d53-962a-ae2ba3b791c6', metadata={'source': 'mammal-pets-doc'}, page_content='Dog are greate companions, know for their loyalty and friendliness.'),
  1.8419873714447021),
 (Document(id='b2f84a37-544b-41b1-9627-f691b50072ad', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  1.912652611732483),
 (Document(id='34ac91a2-3ee2-4ca9-90a7-361cf5705739', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
  1.9452812671661377),
 (Document(id='fa2a432a-58fa-4aeb-98db-21875d1c07cf', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
  1.9491221904754639)]

## Retrievers

Langchain vectorStore objects do not subclass Runnable, and so cannot immediately be intergrated into langchain expression language chains.

langchain Retrievers are runnables, so they implement a standard set of methods (eg. synchronous and asynchronous invoke and batch operations) and are designed to be incorporated in lcel chains. 

we can create a simple version of this ourselves, without subclassing retriever. if we choose what method we wish to use to retrieve documents, we can create a runnable easily.Below we will build one around the similarity_search method.

In [23]:
from typing import List 

from langchain_core.runnables import RunnableLambda

retriever=RunnableLambda(vectorStore.similarity_search).bind(k=1)
retriever.batch(["Cat", "dog"])

[[Document(id='a7fe7797-bfab-4d53-962a-ae2ba3b791c6', metadata={'source': 'mammal-pets-doc'}, page_content='Dog are greate companions, know for their loyalty and friendliness.')],
 [Document(id='a7fe7797-bfab-4d53-962a-ae2ba3b791c6', metadata={'source': 'mammal-pets-doc'}, page_content='Dog are greate companions, know for their loyalty and friendliness.')]]

Vectorstores implement an as_retriever method that will generate a Retriever, specifically a VectorStoreRetriever. These retrievers include specific search_type and search_kwargs attributes that identify what methods of the underlying vector store to call, and how to parameterize them. For instance,we can replicate the above with the following:

In [24]:
retriever=vectorStore.as_retriever(
    search_type="similarity",
    search_kwargs={"k":1}
) 

In [25]:
retriever.batch(["cat","dog"])

[[Document(id='a7fe7797-bfab-4d53-962a-ae2ba3b791c6', metadata={'source': 'mammal-pets-doc'}, page_content='Dog are greate companions, know for their loyalty and friendliness.')],
 [Document(id='a7fe7797-bfab-4d53-962a-ae2ba3b791c6', metadata={'source': 'mammal-pets-doc'}, page_content='Dog are greate companions, know for their loyalty and friendliness.')]]

In [30]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message="""
Answer this question using the provided context only.

{question}

context :
{context}
"""

In [31]:
prompt=ChatPromptTemplate.from_messages([("human",message)])

In [32]:
rag_chain={"context":retriever,"question":RunnablePassthrough()}|prompt|llm

In [33]:
response=rag_chain.invoke("tell me about dogs")

In [34]:
print(response)

AIMessage(
    content='Dogs are great companions, known for their loyalty and friendliness.',
    additional_kwargs={'refusal': None},
    response_metadata={
        'token_usage': {
            'completion_tokens': 149,
            'prompt_tokens': 80,
            'total_tokens': 229,
            'completion_tokens_details': {
                'accepted_prediction_tokens': 0,
                'audio_tokens': 0,
                'reasoning_tokens': 128,
                'rejected_prediction_tokens': 0
            },
            'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
        },
        'model_name': 'gpt-5-2025-08-07',
        'system_fingerprint': None,
        'id': 'chatcmpl-C8UlrcSxCMjCtRkyMCDWIwKAd0Nmw',
        'service_tier': 'default',
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run--59510cf3-031c-4fe7-9f97-62d9426608e1-0',
    usage_metadata={
        'input_tokens': 80,
        'output_tokens': 149,
        'total_tokens': 229,
        'input_token_details': {'audio': 0, 'cache_read': 0},
        'output_token_details': {'audio': 0, 'reasoning': 128}
    }
)